In [ ]:
# SETUP
import random
import numpy as np
import csv
import tensorflow as tf
import tensorflow.keras as keras
import itertools
import math
import time
import tensorflow_probability as tfp
import functools

pi = math.pi

In [ ]:
# GENERATING DATA (AND WRITE TO .CSV)
def flatArr(x):
    y = []
    for row in x:
        for el in row:
            y.append(el)
    return np.array(y).flatten()

def sampling(x_range, y_range, t_range, t_size, int_size, bound_size):
    int_x = np.random.uniform(x_range[0], x_range[1], int_size)
    int_y = np.random.uniform(y_range[0], y_range[1], int_size)
    Boundary_points = []
    for i in range(bound_size):
        random_point = [random.uniform(x_range[0], x_range[1]), random.uniform(y_range[0], y_range[1])]
        random_index = random.randint(0,1)
        if random_index == 0:
            random_value = random.choice([x_range[0], x_range[1]])
        else:
            random_value = random.choice([y_range[0], y_range[1]])
        random_point[random_index] = random_value
        Boundary_points.append(random_point)
    bound_x = np.array(Boundary_points)[:, 0]
    bound_y = np.array(Boundary_points)[:, 1]
    delta_t = t_range[1]/t_size
    int_x_t = np.array([])
    int_y_t = np.array([])
    bound_x_t = np.array([])
    bound_y_t = np.array([])
    t = []
    tb = []
    for i in range(t_size):
        t.append([round(t_range[0]+(i+1)*delta_t,6) for e in range(int_x.size)])
        int_x_t = np.concatenate((int_x_t, int_x))
        int_y_t = np.concatenate((int_y_t, int_y))
        tb.append([round(t_range[0]+(i+1)*delta_t,6) for e in range(bound_x.size)])
        bound_x_t = np.concatenate((bound_x_t, bound_x))
        bound_y_t = np.concatenate((bound_y_t, bound_y))
    t = flatArr(t)
    tb = flatArr(tb)
    # initial points----------------------------------------------
    int_x = np.random.uniform(x_range[0], x_range[1], int(int_x_t.size/2))
    int_y = np.random.uniform(y_range[0], y_range[1], int(int_x_t.size/2))
    Boundary_points = []
    for i in range(int(int_x_t.size/2)):
        random_point = [random.uniform(x_range[0], x_range[1]), random.uniform(y_range[0], y_range[1])]
        random_index = random.randint(0,1)
        if random_index == 0:
            random_value = random.choice([x_range[0], x_range[1]])
        else:
            random_value = random.choice([y_range[0], y_range[1]])
        random_point[random_index] = random_value
        Boundary_points.append(random_point)
    bound_x = np.array(Boundary_points)[:, 0]
    bound_y = np.array(Boundary_points)[:, 1]
    int_x = np.concatenate((int_x, bound_x))
    int_y = np.concatenate((int_y, bound_y))
    return int_x, int_y, t, int_x_t, int_y_t, tb, bound_x_t, bound_y_t

# training data
N_train = 1000;        # number of points
int_x, int_y, t, int_x_t, int_y_t, tb, bound_x_t, bound_y_t = sampling([0.0, 1.0], [0.0, 1.0], [0.0, 1.0], 10, N_train, N_train)

# write training data to .csv file
# with open('/content/drive/MyDrive/Papers/Code (chung)/data_train.csv', mode='w') as f:
with open('data_train.csv', mode='w') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for i in range(t.size):
        csv_writer.writerow([int_x_t[i], int_y_t[i], t[i], bound_x_t[i], bound_y_t[i], tb[i], int_x[i], int_y[i], 0.0])


# test data
N_test = 200;        # number of points
int_x, int_y, t, int_x_t, int_y_t, tb, bound_x_t, bound_y_t = sampling([0.0, 1.0], [0.0, 1.0], [0.0, 1.0], 5, N_test, N_test)

# write test data to .csv file
# with open('/content/drive/MyDrive/Papers/Code (chung)/data_test.csv', mode='w') as f:
with open('data_test.csv', mode='w') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for i in range(t.size):
        csv_writer.writerow([int_x[i], int_y[i], 0.0])
    for i in range(t.size):
        csv_writer.writerow([int_x_t[i], int_y_t[i], t[i]])
    for i in range(t.size):
        csv_writer.writerow([bound_x_t[i], bound_y_t[i], tb[i]])

In [ ]:
# READ DATA

# load the dataset
# data_train = np.float32(np.loadtxt('/content/drive/MyDrive/Papers/Code (chung)/data_train.csv', delimiter=','))
# data_test = np.float32(np.loadtxt('/content/drive/MyDrive/Papers/Code (chung)/data_test.csv', delimiter=','))

data_train = np.float32(np.loadtxt('data_train.csv', delimiter=','))
data_test = np.float32(np.loadtxt('data_test.csv', delimiter=','))
# data_train_test = np.float32(np.loadtxt('data_train_test.csv', delimiter=','))
# # split into datasets
# P_in = dataset[:,0:2]
# P_b = dataset[:,2:4]

# # Load NumPy arrays with tf.data.Dataset
# P_in = tf.data.Dataset.from_tensor_slices(P_in)
# P_b = tf.data.Dataset.from_tensor_slices(P_b)

# # Shuffle and batch the datasets
# BATCH_SIZE=32
# SHUFFLE_BUFFER_SIZE=100

# P_in = P_in.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
# P_b = P_b.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)

In [ ]:
# BUILD NETWORK

# dimension of input and output
in_dim = 3
out_dim = 3

# number of neurons on each layer
nn = [32, 32, 32, 32, 32]

# input layer
inputs = keras.Input(shape=(in_dim,), name='points')

# hidden layers
hidden = keras.layers.Dense(nn[0], activation='tanh', name='hidden_1')(inputs)
for i in range(len(nn)-1):
    hidden = keras.layers.Dense(nn[i+1], activation='tanh', name='hidden_' + str(i+2))(hidden)

# output layer
outputs = keras.layers.Dense(out_dim, activation='linear', name="u")(hidden)

# create network
PDEmodel = keras.Model(inputs=inputs, outputs=outputs, name='chorinvorte')

# show network details
PDEmodel.summary()

Model: "chorinvorte"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
points (InputLayer)          [(None, 3)]               0         
_________________________________________________________________
hidden_1 (Dense)             (None, 32)                128       
_________________________________________________________________
hidden_2 (Dense)             (None, 32)                1056      
_________________________________________________________________
hidden_3 (Dense)             (None, 32)                1056      
_________________________________________________________________
hidden_4 (Dense)             (None, 32)                1056      
_________________________________________________________________
hidden_5 (Dense)             (None, 32)                1056      
_________________________________________________________________
u (Dense)                    (None, 3)                 

In [ ]:
# TRAINING

# GRAD U
def grad_u(X):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(X)
        uvp = PDEmodel(X)
        u = uvp[:, 0]
        v = uvp[:, 1]
        p = uvp[:, 2]

    u_t = tape.gradient(u, X)[:, 2]
    u_x = tape.gradient(u, X)[:, 0]
    u_y = tape.gradient(u, X)[:, 1]
    v_t = tape.gradient(v, X)[:, 2]
    v_x = tape.gradient(v, X)[:, 0]
    v_y = tape.gradient(v, X)[:, 1]
    p_x = tape.gradient(p, X)[:, 0]
    p_y = tape.gradient(p, X)[:, 1]

    return u, v, u_t, u_x, u_y, v_t, v_x, v_y, p_x, p_y

# return 1D weights
def get_weights():
    w = []
    for layer in PDEmodel.layers[1:]:
        weights_biases = layer.get_weights()
        weights = weights_biases[0].flatten()
        biases = weights_biases[1]
        w.extend(weights)
        w.extend(biases)
    return w

# get size of weights in each model's layer
sizes_w = []
sizes_b = []
for layer in PDEmodel.layers[1:]:
    weights_biases = layer.get_weights()
    sizes_w.append(weights_biases[0].flatten().size)
    sizes_b.append(weights_biases[1].size)

# convert 1D weights to multi dimension weights in each model's layer
def set_weights(w):
    for i, layer in enumerate(PDEmodel.layers[1:]):
        start_weights = sum(sizes_w[:i]) + sum(sizes_b[:i])
        end_weights = sum(sizes_w[:i+1]) + sum(sizes_b[:i])
        weights = w[start_weights:end_weights]
        w_div = int(sizes_w[i] / sizes_b[i])
        weights = tf.reshape(weights, [w_div, sizes_b[i]])
        biases = w[end_weights:end_weights + sizes_b[i]]
        weights_biases = [weights, biases]
        layer.set_weights(weights_biases)

losses_hist = []
def train(epochs, BATCH_SIZE=128, SHUFFLE_BUFFER_SIZE=100, optimizer = 'adam'):
    Re = 100.0
    # split into datasets
    P_in = data_train[:,0:3]
    # P_in_i = P_in[0:N_train]
    # P_in = P_in[N_train:]
    P_b = data_train[:,3:6]
    # P_b_i = P_b[0:N_train]
    # P_b = P_b[N_train:]
    # P_i = np.concatenate((P_in_i, P_b_i))
    P_i = data_train[:, 6:9]
    # P_i_tf = tf.convert_to_tensor(P_i, dtype=tf.float32)
    # CHOOSE OPTIMIZER
    if (optimizer == 'adam'):
        optimizer = tf.keras.optimizers.Adam(beta_1=0.9, beta_2=0.999)
        # Load NumPy arrays with tf.data.Dataset
        P_in = tf.data.Dataset.from_tensor_slices(P_in)
        P_b = tf.data.Dataset.from_tensor_slices(P_b)
        P_i = tf.data.Dataset.from_tensor_slices(P_i)
        # Shuffle and batch the datasets
        P_in = P_in.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
        P_b = P_b.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
        P_i = P_i.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
        # for loop epochs
        for epoch in range(epochs):
            start_epoch = time.time()
            # for loop iteration (all batches)
            loss = [0]
            for (P_in_batch, P_b_batch, P_i_batch) in itertools.zip_longest(P_in, P_b, P_i):
                # interior loss
                with tf.GradientTape() as tape:
                    with tf.GradientTape(persistent=True) as tape1:
                        tape1.watch(P_in_batch)
                        u, v, u_t, u_x, u_y, v_t, v_x, v_y, p_x, p_y = grad_u(P_in_batch)

                    # u_xx and u_yy
                    u_xx = tape1.gradient(u_x, P_in_batch)[:,0]
                    u_yy = tape1.gradient(u_y, P_in_batch)[:,1]
                    v_xx = tape1.gradient(v_x, P_in_batch)[:,0]
                    v_yy = tape1.gradient(v_y, P_in_batch)[:,1]
                    
                    # loss_in_batch
                    f_u = u_t - 1/Re*(u_xx + u_yy) + u*u_x + v*u_y + p_x
                    f_v = v_t - 1/Re*(v_xx + v_yy) + u*v_x + v*v_y + p_y
                    div_u = u_x + v_y
                    loss_in_batch = tf.reduce_mean(tf.math.square(f_u)) + \
                                    tf.reduce_mean(tf.math.square(f_v)) + \
                                    tf.reduce_mean(tf.math.square(div_u))
                    
                    # loss_b_batch
                    uvp = PDEmodel(P_b_batch)
                    u = uvp[:,0]
                    v = uvp[:,1]
                    loss_b_batch = tf.reduce_mean(tf.math.square(u + tf.math.cos(pi*P_b_batch[:,0])*tf.math.sin(pi*P_b_batch[:,1])*tf.math.exp(-2.0*pi*pi*P_b_batch[:,2]/Re))) + \
                                   tf.reduce_mean(tf.math.square(v - tf.math.sin(pi*P_b_batch[:,0])*tf.math.cos(pi*P_b_batch[:,1])*tf.math.exp(-2.0*pi*pi*P_b_batch[:,2]/Re)))
                    
                    # loss_i_batch
                    # if (P_i_batch == None):
                    #     P_i_batch = P_i_batch_None
                    #     P_i_batch = P_i_tf
                    uvp = PDEmodel(P_i_batch)
                    u = uvp[:,0]
                    v = uvp[:,1]
                    loss_i_batch = tf.reduce_mean(tf.math.square(u+tf.math.cos(pi*P_i_batch[:,0])*tf.math.sin(pi*P_i_batch[:,1]))) + \
                                   tf.reduce_mean(tf.math.square(v-tf.math.sin(pi*P_i_batch[:,0])*tf.math.cos(pi*P_i_batch[:,1])))
                    # P_i_batch_None = P_i_batch

                    # total loss
                    loss_batch = loss_in_batch + loss_b_batch + loss_i_batch
                
                # update paremeters
                grads = tape.gradient(loss_batch, PDEmodel.weights)
                optimizer.apply_gradients(zip(grads, PDEmodel.weights))
                
                # add loss_batch to loss
                loss += loss_batch
            
            loss = loss/len(list(P_in))
            print("[%4s] loss = %12.5f \t %4.3fs" % (epoch, loss, time.time() - start_epoch))
            losses_hist.append(loss.numpy()[0])

            # break condition
            # if (epoch > 200):
            #     if (losses_hist[epoch][0]/losses_hist[epoch-100][0] > 0.9):
            #         break

    if (optimizer == 'l-bfgs'):
        P_in = tf.convert_to_tensor(P_in, dtype=tf.float32)
        P_b = tf.convert_to_tensor(P_b, dtype=tf.float32)
        P_i = tf.convert_to_tensor(P_i, dtype=tf.float32)
        def function_factory(P_in, P_b, P_i):

            def loss_grad(weights):
                start_epoch = time.time()
                with tf.GradientTape() as tape:
                    with tf.GradientTape(persistent=True) as tape1:
                        set_weights(weights)
                        tape1.watch(P_in)
                        u, v, u_t, u_x, u_y, v_t, v_x, v_y, p_x, p_y = grad_u(P_in)

                    # u_xx and u_yy
                    u_xx = tape1.gradient(u_x, P_in)[:,0]
                    u_yy = tape1.gradient(u_y, P_in)[:,1]
                    v_xx = tape1.gradient(v_x, P_in)[:,0]
                    v_yy = tape1.gradient(v_y, P_in)[:,1]

                    # loss_in
                    # - pi/2*tf.math.sin(2*pi*P_in[:,1])*tf.math.exp(-4*pi*pi*P_in[:,0]/Re)
                    # - pi/2*tf.math.sin(2*pi*P_in[:,2])*tf.math.exp(-4*pi*pi*P_in[:,0]/Re)
                    f_u = u_t - (1.0/Re)*(u_xx + u_yy) + u*u_x + v*u_y + p_x 
                    f_v = v_t - (1.0/Re)*(v_xx + v_yy) + u*v_x + v*v_y + p_y 
                    div_u = u_x + v_y
                    loss_in = tf.reduce_mean(tf.math.square(f_u)) + \
                              tf.reduce_mean(tf.math.square(f_v)) + \
                              tf.reduce_mean(tf.math.square(div_u))

                    # loss_b
                    uvp = PDEmodel(P_b)
                    u = uvp[:,0]
                    v = uvp[:,1]
                    loss_b = tf.reduce_mean(tf.math.square(u + tf.math.cos(pi*P_b[:,0])*tf.math.sin(pi*P_b[:,1])*tf.math.exp(-2.0*pi*pi*P_b[:,2]/Re))) + \
                             tf.reduce_mean(tf.math.square(v - tf.math.sin(pi*P_b[:,0])*tf.math.cos(pi*P_b[:,1])*tf.math.exp(-2.0*pi*pi*P_b[:,2]/Re)))

                    # loss_i
                    uvp = PDEmodel(P_i)
                    u = uvp[:,0]
                    v = uvp[:,1]
                    loss_i = tf.reduce_mean(tf.math.square(u+tf.math.cos(pi*P_i[:,0])*tf.math.sin(pi*P_i[:,1]))) + \
                             tf.reduce_mean(tf.math.square(v-tf.math.sin(pi*P_i[:,0])*tf.math.cos(pi*P_i[:,1])))

                    # total loss
                    loss = loss_in + loss_b + loss_i
                    print("loss = %12.5f \t %4.3fs" % (loss, time.time() - start_epoch))
                    losses_hist.append(loss.numpy())
                grad = tape.gradient(loss, PDEmodel.weights)
                grad_1D = []
                for g in grad:
                    grad_1D.append(tf.reshape(g, [-1]))
                grad_1D = tf.concat(grad_1D, 0)
                return loss, grad_1D
            return loss_grad
        # update paremeters

        func = function_factory(P_in, P_b, P_i)

        # @tf.function
        # def epoch():
        #     if opt_result is None:
        #         return 0
        #     return int(opt_result.num_iterations.numpy())
        # add loss_batch to loss
        opt_result = tfp.optimizer.lbfgs_minimize(func,
            tf.convert_to_tensor(get_weights(), dtype=tf.float32),
            max_iterations=epochs)
    
        print(opt_result)
        # set_weights(result)
        # print(result)

        # start = get_weights()
        # @tf.function
        # def opt_with_bfgs():
        #     return tfp.optimizer.lbfgs_minimize(
        #         loss_grad,
        #         initial_position=tf.convert_to_tensor(start, dtype=tf.float32),
        #         max_iterations=epochs)

        # results = run(opt_with_bfgs)
        # print('BFGS Results')
        # print('Converged:', results.converged)
        # print('Location of the minimum:', results.position)
        # print('Number of iterations:', results.num_iterations)
    

    

In [ ]:
train(200, optimizer='adam')
train(400, optimizer='l-bfgs')

In [ ]:
def u(network,x,y,t):
    z = tf.convert_to_tensor([[x, y, t]])
    uvp = network(z)
    u = float(uvp[:,0])
    v = float(uvp[:,1])
    return u, v

def exact_sol(x,y,t):
    Re = 100.0
    u = - math.cos(pi*x)*math.sin(pi*y)*math.exp(-2.0*pi*pi*t/Re)
    v = math.sin(pi*x)*math.cos(pi*y)*math.exp(-2.0*pi*pi*t/Re)
    return u, v

P = data_test

err = 0
for i in range(len(P)):
    u_pred, v_pred = u(PDEmodel, P[i][0], P[i][1], P[i][2])
    u_e, v_e = exact_sol(P[i][0], P[i][1], P[i][2])
    err += (u_pred - u_e)**2 + (v_pred - v_e)**2
    # if (i < 1000):
    # print(u_e)
    # print(u_pred)
    # print("-----")

L2 = math.sqrt(err/len(P))
print("L2 = ", L2)

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
losses = np.array(losses_hist)
losses = losses.reshape(len(losses))
epochs = losses.size
x_epochs = [i+1 for i in range(epochs)]

fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(x_epochs,losses, color = 'blue')
ax.yaxis.set_major_formatter(mtick.FormatStrFormatter('%.2e'))
plt.setp(ax, xlabel='Iteration')
plt.setp(ax, ylabel='Loss')
plt.show()

loss_file = open("Loss.txt", "w")
for row in np.array(losses_hist).reshape(len(losses_hist), 1):
    np.savetxt(loss_file, row)
loss_file.close()